In [ ]:
!conda install -y line_profiler

In [3]:
import numpy
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

In [524]:
%load_ext line_profiler

In [4]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [5]:
dict = unpickle('dataset/data_batch_1')
print(list(dict.keys()))
print(dict[b'batch_label'])
print(dict[b'filenames'].__len__())

print(dict[b'data'][0])
print(dict[b'labels'][0])
print(dict[b'data'][0].shape)
#print(dict[b'data'][0].resahpe(32, 32))


[b'batch_label', b'labels', b'data', b'filenames']
b'training batch 1 of 5'
10000
[ 59  43  50 ... 140  84  72]
6
(3072,)


In [6]:
p = Image.fromarray(dict[b'data'][0])

In [7]:
from PIL import Image
import numpy as np
im = Image.open('sample.jpg')
a = np.asarray(im)

FileNotFoundError: [Errno 2] No such file or directory: 'sample.jpg'

In [ ]:

def cifar10_plot(data, img_index=1):
    im = data[b'data'][img_index, :]

    im_r = im[0:1024].reshape(32, 32)
    im_g = im[1024:2048].reshape(32, 32)
    im_b = im[2048:].reshape(32, 32)

    # this is the funciton I was looking for the whole time...
    img = np.dstack((im_r, im_g, im_b))

    print("shape: ", img.shape)
    print("label: ", data[b'labels'][img_index])
    
    #print("category:", meta[b'label_names'][data[b'labels'][im_idx]])         

    plt.imshow(img)
    plt.show()


cifar10_plot(dict)

# Assignment #1 

train set: data_batch_1

validation set: data_batch_2

test set: test_batch

In [583]:

def unpickle(file, n=10000):
    """
    Parameters: 
    file (str): file address 
    num (int): number of datapoints in the file
    """
    import pickle
    import copy 

    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    
    # let's make the Y (which is a K*n matrix of the one-hot representations of the label of each image)
    Y = np.zeros((10, n))
    zero_list = [0 for i in range(10)]
    i = 0
    for a in dict[b'labels']:
        '''
        right = str(bin(a+1)).split('b')[1]
        one_hot = np.array(list('0'*(10-len(right))+right))
        '''
        one_hot = copy.deepcopy(zero_list)
        one_hot[int(a)] = 1
        Y[:, i] = one_hot
        i += 1
        
        

    return dict[b'data'].T, Y, np.array(dict[b'labels'])

train_X, train_Y, train_y = unpickle('dataset/data_batch_1')


In [584]:
def normalize(data_):
    """
    This is used to normalize the data w.r.t mean and std: (x - mean) / std
    """
    data = np.copy(data_)
    shape = data.shape
    mean = np.mean(data, 1)
    std = np.std(data, 1)


    for i in range(data.shape[1]):
        data[:, i] = (data[:, i] - mean) / std

    return data
    
n_train_X = normalize(train_X)

In [585]:
train_X.shape

(3072, 10000)

In [586]:
n_train_X.shape

(3072, 10000)

In [587]:
print(train_X[0, 0])

59


In [588]:
print(n_train_X[100, 4])

0


In [589]:
print(n_train_X[0, 0])

0


In [590]:
print(train_X[0, 0])

59


In [591]:
print(np.mean(train_X, 1)[1])
print(np.std(train_X, 1)[1])

131.3316
72.79520891816986


In [592]:
print(n_train_X[0, 0])

0


In [593]:
train_X

array([[ 59, 154, 255, ...,  71, 250,  62],
       [ 43, 126, 253, ...,  60, 254,  61],
       [ 50, 105, 253, ...,  74, 211,  60],
       ...,
       [140, 139,  83, ...,  68, 215, 130],
       [ 84, 142,  83, ...,  69, 255, 130],
       [ 72, 144,  84, ...,  68, 254, 131]], dtype=uint8)

In [843]:
class MiniBatchGDNN:
    
    def __init__(self, k, d):
        # k is the number of classes
        # d is the data dimention
        self.k = k
        self.d = d

        # initializing the weight matrix
        self.W = np.random.normal(0, 0.01, (k, d))
        self.b = np.random.normal(0, 0.01, (k, 1))

    
    def evaluate_classifier(self, X, W, b):
        # Each column in X corresponds to 'one' image in this context having the d*n size
        # This function returns: k*n, each row contains probability for the specific class

        if len(X.shape) >= 2:
            second_dimention = X.shape[1]
            temp_b = np.array(b * second_dimention)
            result = np.dot(W, X) + temp_b
        else:
            second_dimention = 0
            temp_b = b
            result = np.dot(W, X).reshape(10, 1) + temp_b
        
        

        P = np.zeros(result.shape).shape
        
        if second_dimention:
            P = np.zeros(result.shape)
            for j in range(second_dimention):
                P[:, j] = np.exp(result[:, j]) / sum(np.exp(result[:, j]))
        else:
            P = np.zeros((result.shape[0], 1))
            P = (np.exp(result) / sum(np.exp(result))).reshape((self.k, 1))
            # just one row of elements in P
        
        return P
    

    def compute_cost(self, X, Y, W, b, lambda_):
        n = X.shape[1]
        r = np.sum(np.square(self.W))
        P = self.evaluate_classifier(X, W, b)
        l = 0
        for i in range(n):
            y = Y[:, i]
            p = P[:, i]
            probability = np.dot(y, p)

            if not probability:
                '''
                print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
                print('y is {0}\n p is: {1}'.format(y, p))
                print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
                '''
                probability = 1.00e-100


            assert probability

            l = - np.log(probability)
            
            if not l:
                print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
                print('probablity: {}'.format(probability))
                print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
            
            assert l
            
        J = (l / n) + (lambda_ * r)
        
        return J
    
    def accuracy(self, X, y, W, b):
        # y is a vector containing the ground truth label numbers (just like train_y)
        P = self.evaluate_classifier(X, W, b)
        prediction = np.argmax(P, axis=0)
        n = X.shape[1]
        incorrect = 0
        for i in range(n):
            if y[i] != prediction[i]:
                incorrect += 1
        return (n - incorrect) / n
    

    def compute_grads_num_slow(self, X, Y, P, W, b, lamda_, h=0.01):
        

        no 	= 	W.shape[0]
        d 	= 	X.shape[0]

        grad_W = np.zeros(W.shape)
        grad_b = np.zeros((no, 1))


        
        for i in range(len(b)):
            b_try = np.array(b)
            b_try[i] -= h
            c1 = self.compute_cost(X, Y, W, b_try, lamda_)

            b_try = np.array(b)
            b_try[i] += h
            c2 = self.compute_cost(X, Y, W, b_try, lamda_)

            grad_b[i] = (c2-c1) / (2*h)
        

        for i in range(W.shape[0]):
            print('>> inside grad computation main loop, i: {}'.format(i))
            for j in range(W.shape[1]):

                W_try = np.array(W)
                W_try[i,j] -= h
                c1 = self.compute_cost(X, Y, W_try, b, lamda_)

                W_try = np.array(W)
                W_try[i,j] += h
                c2 = self.compute_cost(X, Y, W_try, b, lamda_)

                grad_W[i,j] = (c2-c1) / (2*h)

                
                if np.isnan(grad_W[i,j]):
                    print('You are in a bad situation...')
                    print(2*h)
                    print(c1)
                    print(c2)
                    print('X: \n {}'.format(X))
                    print('Y: \n {}'.format(Y))
                    print('Is W_try nan: {0}\nW_try: \n {1}'.format(np.isnan(np.sum(W_try)), W_try))
                    print('Is b nan: {0}\nb: \n {1}'.format(np.isnan(np.sum(b)), b))
                    print('lambda: {}'.format(lamda_))
                    print('#####################')
                
                

        if np.isnan(np.sum(grad_W)):
            print('\n.   ^^^^^^^^^^^^^^^^^^^^ smt bad: ')
        
        assert not np.isnan(np.sum(grad_W))

        return [grad_W, grad_b]



    def train(self, X, Y, eta=0.01, lambda_=0, epochs=100, mini_batch_size=100):
        # forward pass
        # backward pass
        # update weight matrix
        
        # let's do a permutation of the total dataset size!
        n = X.shape[1]
        assert n % mini_batch_size == 0
        for i in range(epochs):
            '''
            # if we use train_y as well:
            temp1 = np.concatenate((train_X, train_Y))
            temp2 = train_y.reshape((train_y.shape[0], 1)).T
            stacked_arrays = np.concatenate((temp1, temp2)).shape
            '''
            stacked_arrays = np.concatenate((X, Y))
            np.take(stacked_arrays, np.random.permutation(n), axis=1, out=stacked_arrays)
            X_, Y_ = np.split(stacked_arrays, [self.d], axis=0)

            num_of_batches = int(n / mini_batch_size)
            l_index = 0
            h_index = 0
            for i in range(num_of_batches):
                l_index = h_index
                h_index = (i+1) * mini_batch_size
                batch_X = X_[:, l_index:h_index]
                batch_Y = Y_[:, l_index:h_index]
                P = self.evaluate_classifier(batch_X, self.W, self.b)

                grad_W, grad_b = self.compute_grads_num_slow(batch_X, batch_Y, P, self.W, self.b, lambda_)
                
                print('******** grads computed, batch number: {}'.format(i))
                
                # TODO: remove this:
                print('shape of W: {}'.format(self.W.shape))
                print('shape of grad_W: {}'.format(grad_W.shape))
                print('shape of b: {}'.format(self.b.shape))
                print('shape of grad_b: {}'.format(grad_b.shape))
                print('this is grad_W:')
                print(grad_W)
                print('dtype of grad_W: {0}, dtype of W: {1}'.format(self.W.dtype, grad_W.dtype))
                print('eta is: {}'.format(eta))
                print('result of operation (temp): ')
                print(self.W - (eta * grad_W))
                '''import time
                time.sleep(10)'''


                self.W = self.W - (eta * grad_W)
                self.b = self.b - (eta * grad_b)

                # TODO: remove this
                print('------------------------')
            
            # TODO: remove this
            print('epoch ended...')



In [844]:
nn = MiniBatchGDNN(k=10, d=5)
#res = nn.evaluate_classifier(train_X[:, 0], nn.W, nn.b)
#nn.compute_cost(n_train_X, train_Y, nn.W, nn.b, 0.01)
#nn.accuracy(n_train_X, train_y, nn.W, nn.b)

In [845]:
%lprun -f nn.train nn.train(n_train_X[0:5, :], train_Y)

>> inside grad computation main loop, i: 0
>> inside grad computation main loop, i: 1
>> inside grad computation main loop, i: 2
>> inside grad computation main loop, i: 3
>> inside grad computation main loop, i: 4
>> inside grad computation main loop, i: 5
>> inside grad computation main loop, i: 6
>> inside grad computation main loop, i: 7
>> inside grad computation main loop, i: 8
>> inside grad computation main loop, i: 9
******** grads computed, batch number: 0
shape of W: (10, 5)
shape of grad_W: (10, 5)
shape of b: (10, 1)
shape of grad_b: (10, 1)
this is grad_W:
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
dtype of grad_W: float64, dtype of W: float64
eta is: 0.01
result of operation (temp): 
[[-0.01449816  0.00221223 -0.00938178  0.00607518  0.01157121]
 [ 0.00837532 -0.00377475  0.01097295 -0.01057078  0.00586676]
 [ 0.0235476   0.00161914 -0

AssertionError: 

In [ ]:
print(nn.W)

In [ ]:
J = nn.compute_cost(n_train_X, train_Y, nn.W, nn.b, 0.01)

242.4582915979156

0


In [444]:
f = np.array([1, 2, 4, 7, 11, 16], dtype=float)
x = np.arange(f.size)

In [445]:
f

array([ 1.,  2.,  4.,  7., 11., 16.])

In [446]:
x

array([0, 1, 2, 3, 4, 5])

In [449]:
np.gradient(f, x)

array([1. , 1.5, 2.5, 3.5, 4.5, 5. ])